In [ ]:
# --- 🚀 TORUS universal imports / helpers ---
import os, json, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats import chi2, pearsonr
from scipy.special import erfcinv
np.random.seed(42)

# Safe σ helper – never let σ==0 blow up χ²
def safe_sigma(sig, floor=1e-12):
    sig = np.asarray(sig, dtype=float)
    bad = sig < floor
    if bad.any():
        print(f'[boot] σ floor applied to {bad.sum()} cells')
        sig[bad] = floor
    return sig


# PairCorr Side-Band Benchmark
Synthetic test: injects a 7 % side-band at $k_1=k_0(1+1/14)$ and runs the TORUS detection pipeline.

In [ ]:
# --- 🚀 TORUS universal imports / helpers ---
import os, json, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats import chi2, pearsonr
from scipy.special import erfcinv
np.random.seed(42)

# Safe σ helper – never let σ==0 blow up χ²
def safe_sigma(sig, floor=1e-12):
    sig = np.asarray(sig, dtype=float)
    bad = sig < floor
    if bad.any():
        print(f'[boot] σ floor applied to {bad.sum()} cells')
        sig[bad] = floor
    return sig


In [ ]:
import numpy as np, matplotlib.pyplot as plt
from scipy.signal import periodogram
try:
    from scipy.signal.windows import hann       # SciPy ≥1.4
except ImportError:
    from numpy import hanning as hann           # fallback for old builds
import scipy, numpy
assert numpy.__version__ < '2.3', "Pin NumPy <2.3 until SciPy wheels catch up"

N, L = 1000, 100
k0 = 2*np.pi/(L/10)
k1 = k0*(1+1/14)
amp1 = 0.07
x = L*np.random.rand(N)
y = L*np.random.rand(N) + amp1*np.sin(k1*x)


In [ ]:
from scipy.spatial.distance import pdist
dists = pdist(np.column_stack([x, y]))
bins = np.linspace(0, L, 200)
g2, _ = np.histogram(dists, bins=bins)
g2 = g2 / g2.max()


In [ ]:
try:
    from scipy.signal.windows import hann     # SciPy ≥ 1.4
except ImportError:
    from numpy import hanning as hann         # fallback for older SciPy
from scipy.signal import periodogram
freqs, power = periodogram(g2 * hann(len(g2)), scaling='spectrum')


In [ ]:
k0_idx = np.argmax(power)
k0_val = freqs[k0_idx]
k1_val = k0_val * (1 + 1/14)
k1_idx = np.argmin(abs(freqs - k1_val))
deltaP = 10*np.log10(power[k1_idx]/power[k0_idx])
print('k0=', k0_val, 'k1=', k1_val, 'ΔP=', deltaP, 'dB')
if abs(k1_val-freqs[k1_idx])/k1_val<=.03 and -45<deltaP<-25:
    print('TORUS-POSITIVE')
else:
    print('TORUS-NEGATIVE')
